In [1]:
#Load and view the data
import numpy as np
import pandas as pd

data=pd.read_csv('C:/Users/xiuro/AppData/Local/Programs/Python/Python311/save/children anemia.csv')
print(data.head())
print("data's info:\n",data.info())
#print("describe of data:\n:", data.describe())
data.replace('',np.nan,inplace=True)
df = data.copy()
df.rename(columns={'Age in 5-year groups':'Age groups','Type of place of residence':'Residence type',
                   'Highest educational level': 'Education Level' , 'Wealth index combined': 'Wealth index','Births in last five years':'Births in last five years',
                   'Age of respondent at 1st birth':'Age of 1st birth',
                   'Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)':'Hemoglobin level-altitude and smoking',
                   'Have mosquito bed net for sleeping (from household questionnaire)':'Mosquito net',
                   'Currently residing with husband/partner':'together with partner',
                   'Hemoglobin level adjusted for altitude (g/dl - 1 decimal)':'Hemoglobin level-altitude',
                   'Taking iron pills, sprinkles or syrup':'Iron pills'}, inplace =True)
print(df.info())                   

# Missing values checking 
missing_values=df.isnull().sum()
print(missing_values)
#duplicated values checking and imputation
duplicated_values= df.duplicated().sum()
print(duplicated_values)
df = df.drop_duplicates()
print(df.info())

  Age in 5-year groups Type of place of residence Highest educational level  \
0                40-44                      Urban                    Higher   
1                35-39                      Urban                    Higher   
2                25-29                      Urban                    Higher   
3                25-29                      Urban                 Secondary   
4                20-24                      Urban                 Secondary   

  Wealth index combined  Births in last five years  \
0               Richest                          1   
1               Richest                          1   
2               Richest                          1   
3               Richest                          1   
4               Richest                          1   

   Age of respondent at 1st birth  \
0                              22   
1                              28   
2                              26   
3                              25   
4              

In [2]:
#missing values imputation 
#after initial analysis, only one set of anemia level and hemoglobin level was decided on for further analysis 
df.drop(['Anemia level_2','Hemoglobin level-altitude'],axis=1, inplace=True)

df2= df.dropna()

print(df2.info())
df_new=df2.copy()

<class 'pandas.core.frame.DataFrame'>
Index: 7273 entries, 3 to 33922
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Age groups                             7273 non-null   object 
 1   Residence type                         7273 non-null   object 
 2   Education Level                        7273 non-null   object 
 3   Wealth index                           7273 non-null   object 
 4   Births in last five years              7273 non-null   int64  
 5   Age of 1st birth                       7273 non-null   int64  
 6   Hemoglobin level-altitude and smoking  7273 non-null   float64
 7   Anemia level                           7273 non-null   object 
 8   Mosquito net                           7273 non-null   object 
 9   Smokes cigarettes                      7273 non-null   object 
 10  Current marital status                 7273 non-null   object 
 11  together

In [3]:
# Label encoding for categorical columns
categorical_columns = df_new.select_dtypes(include='object').columns

from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
for col in categorical_columns:
    df_new[col]=labelencoder.fit_transform(df_new[col])

print(df_new.info())


<class 'pandas.core.frame.DataFrame'>
Index: 7273 entries, 3 to 33922
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Age groups                             7273 non-null   int32  
 1   Residence type                         7273 non-null   int32  
 2   Education Level                        7273 non-null   int32  
 3   Wealth index                           7273 non-null   int32  
 4   Births in last five years              7273 non-null   int64  
 5   Age of 1st birth                       7273 non-null   int64  
 6   Hemoglobin level-altitude and smoking  7273 non-null   float64
 7   Anemia level                           7273 non-null   int32  
 8   Mosquito net                           7273 non-null   int32  
 9   Smokes cigarettes                      7273 non-null   int32  
 10  Current marital status                 7273 non-null   int32  
 11  together

In [4]:
#Perform the Shapiro-Wilk test for the continuous numerical variable
from scipy import stats
statistic, p_value = stats.shapiro(df_new['Hemoglobin level-altitude and smoking'])
print(f"Shapiro-Wilk Statistic: {statistic}")
print(f"P-value: {p_value}")

Shapiro-Wilk Statistic: 0.9889780282974243
P-value: 2.2542255724767688e-23


C:\Users\xiuro\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [5]:
#Split Data into Training and Testing Sets
from sklearn.model_selection import train_test_split
X = df_new.drop(columns=['Anemia level'])
y = df_new['Anemia level']
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [6]:
#based the result, choose AdaBoostClassifier for further analysis 
from sklearn.ensemble import AdaBoostClassifier
# Create the AdaBoostClassifier
ada_classifier = AdaBoostClassifier(n_estimators=100, random_state=42)

# Train the classifier
ada_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = ada_classifier.predict(X_test)

In [7]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred) # For classification tasks
report = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(report)

#Get the feature names (column names)
feature_names = X.columns
#Access feature importance using the feature_importances_ attribute of the trained model.
feature_importance = ada_classifier.feature_importances_

feature_imp_df = pd.DataFrame({'feature names':feature_names,'feature importance':feature_importance})
feature_imp_df.sort_values(by='feature importance',ascending=False, inplace=True)
print(feature_imp_df)

Accuracy: 0.9161512027491409
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       373
           1       0.87      0.99      0.93       436
           2       1.00      0.92      0.96       624
           3       0.81      1.00      0.90        22

    accuracy                           0.92      1455
   macro avg       0.88      0.93      0.91      1455
weighted avg       0.92      0.92      0.92      1455

                            feature names  feature importance
6   Hemoglobin level-altitude and smoking                 1.0
0                              Age groups                 0.0
1                          Residence type                 0.0
2                         Education Level                 0.0
3                            Wealth index                 0.0
4               Births in last five years                 0.0
5                        Age of 1st birth                 0.0
7                            Mosquito ne